In [1]:
import time
start_time = time.time()
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()

In [15]:
# Standard imports
import numpy as np
import pandas as pd
import time
import warnings

# Qiskit 
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, EfficientSU2, TwoLocal
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_algorithms.utils import algorithm_globals

# Visualization libraries
from matplotlib import pyplot as plt
import seaborn as sns
from qiskit.visualization import plot_histogram
from IPython.display import clear_output

# Machine learning and preprocessing
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.exceptions import ConvergenceWarning
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [7]:
# -*- coding: utf-8 -*-

# This code is part of Qiskit.
#
# (C) Copyright IBM 2018, 2020.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""
iris dataset
"""

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA


def iris(training_size, test_size, n, plot_data=False):
    """ returns iris dataset """
    class_labels = [r'A', r'B', r'C']
    data, target = datasets.load_iris(return_X_y=True)
    sample_train, sample_test, label_train, label_test = \
        train_test_split(data, target, test_size=1, random_state=42)

    # Now we standardize for gaussian around 0 with unit variance
    std_scale = StandardScaler().fit(sample_train)
    sample_train = std_scale.transform(sample_train)
    sample_test = std_scale.transform(sample_test)

    # Now reduce number of features to number of qubits
    pca = PCA(n_components=n).fit(sample_train)
    sample_train = pca.transform(sample_train)
    sample_test = pca.transform(sample_test)

    # Scale to the range (-1,+1)
    samples = np.append(sample_train, sample_test, axis=0)
    minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
    sample_train = minmax_scale.transform(sample_train)
    sample_test = minmax_scale.transform(sample_test)

    # Pick training size number of samples from each distro
    training_input = {key: (sample_train[label_train == k, :])[:training_size]
                      for k, key in enumerate(class_labels)}
    test_input = {key: (sample_test[label_test == k, :])[:test_size]
                  for k, key in enumerate(class_labels)}

    if plot_data:
        try:
            import matplotlib.pyplot as plt
        except ImportError:
            raise NameError('Matplotlib not installed. Please install it before plotting')
        for k in range(0, 3):
            plt.scatter(sample_train[label_train == k, 0][:training_size],
                        sample_train[label_train == k, 1][:training_size])

        plt.title("Iris dataset")
        plt.show()

    return sample_train, training_input, test_input, class_labels

In [39]:
feature_dim = 4 # dimension of each data point
training_dataset_size = 120
testing_dataset_size = 30
random_seed = 10598
algorithm_globals.random_seed = 4701

sample_Total, training_input, test_input, class_labels = iris(
    training_size=training_dataset_size,
    test_size=testing_dataset_size,
    n=feature_dim,
    plot_data=False
)

In [80]:
data = training_input 

In [81]:
all_data = []

for class_name, features in data.items():
    # Create a column for the class labels
    class_column = np.full((features.shape[0], 1), class_name)
    # Concatenate the class column with the features
    class_features = np.hstack((features, class_column))
    # Append to the master list
    all_data.append(class_features)

# Combine all class features into a single array
combined_data = np.vstack(all_data)

In [86]:
# Correcting the type conversion to avoid the AttributeError

# Extract the features (first 4 columns) and convert them to np.float64
features = combined_data[:, :4].astype(np.float64)  # Use np.float64 for numerical operations

# Extract the class labels (last column) and ensure it's a 2D array of shape (150, 1)
class_labels = combined_data[:, 4].reshape(-1, 1)


In [45]:
optimizer=COBYLA(maxiter=100)
feature_map = ZZFeatureMap(feature_dimension=4)

In [46]:
sampler = Sampler()

In [49]:
    vqc = VQC(
        sampler = sampler, 
        feature_map = feature_map,
        ansatz = None, 
        optimizer = optimizer,)

In [68]:
sample_train

NameError: name 'sample_train' is not defined

In [87]:
vqc.fit(features, class_labels)

In [116]:
vqc.score

<bound method NeuralNetworkClassifier.score of <qiskit_machine_learning.algorithms.classifiers.vqc.VQC object at 0x18b9af080>>

In [103]:
from qiskit_ibm_backend import IBMBackend

ModuleNotFoundError: No module named 'qiskit_ibm_backend'

In [112]:
backends = provider.backends()

In [114]:
least_busy(backends)

<IBMBackend('ibmq_qasm_simulator')>

NameError: name 'Backends' is not defined

In [91]:
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Sampler
from qiskit_machine_learning.algorithms import VQC
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_provider import least_busy
# Example feature map and ansatz
feature_map = ZZFeatureMap(feature_dimension=2)
ansatz = RealAmplitudes(num_qubits=2, reps=1)

# Select a backend

backend = provider.get_backend('ibmq_manila')

# Initialize the Sampler
sampler = Sampler(backend=backend)

# Initialize VQC
vqc = VQC(feature_map=feature_map,
          ansatz=ansatz,
          optimizer=COBYLA(),
          sampler=sampler)

# Example data (X: features, Y: labels)
X = np.array([[0.1, 0.2], [0.2, 0.3], [0.3, 0.4]])
Y = np.array([[1, 0], [1, 0], [0, 1]])

# Fit the model
vqc.fit(X, Y)


QiskitBackendNotFoundError: 'No backend matches the criteria'